Report sau 2 ngày tìm kiếm: Tối thứ 6, đêm T6-sáng T7, tối T7, đêm T7-sáng CN

1, Project Github https://github.com/Syun1208/scence-text-paddle-ocr/ (ông này git clone từ 1 cái được ông tạo sẵn https://github.com/hungcao0402/PaddleOCR-Vietnamese - được rút gọn từ https://github.com/PaddlePaddle/PaddleOCR ) train tiếp Det SARN và Rec SAR from. 

- Họ sử dụng bộ data Tiếng việt được label sẵn bởi VinAI: https://github.com/VinAIResearch/dict-guided#dataset. (hoặc link ở Drive: https://drive.google.com/drive/folders/1uF3OKxW21NVdmMfQuE7Xh8_wuIjaSZP4?usp=sharing  )

Bài trên blog: http://tutorials.aiclub.cs.uit.edu.vn/index.php/2022/04/20/nhan-dang-chu-tieng-viet-trong-anh-ngoai-canh/).
Video thuyết trinh demo sp: https://www.youtube.com/watch?v=0H3ABACbTU0
Facebook ông đó https://www.facebook.com/syun128/

Tụi em tìm được code train Model PaddleOCR nhận dạng được chữ tiếng việt (HỌ DEMO: nhận diện chữ tiếng việt tốt trên cả biến báo và chữ đại học bách khoa trên tảng đá trường em, như trong ảnh)

2. https://github.com/Syun1208/Smart_menu_OCR - file `model_tect.onnx`, 'model_reg.onnx' được đóng gói để Detect và Recog CHỮ TIẾNG VIỆT. 
(https://github.com/Syun1208/Smart_menu_OCR mở mục models) (cũng trên github của ông này). (Demo là ảnh hóa đơn quán Lẩu Nướng)

- Em thử chạy test với ảnh thì bị lỗi BUG 
```
Name:'Add_44' Status Message: C:\a\_work\1\s\onnxruntime\core/providers/cpu/math/element_wise_ops.h:540 onnxruntime::BroadcastIterator::Init axis == 1 || axis == largest was false. Attempting to broadcast an axis by a dimension other than 1. 319 by 320
```
(lỗi liên quan đến việc 2 tensor đầu vào của 1 nút có kích thước không tương thích trên cùng 1 trục, và đều khác 1 nên broadcast cũng ko thể xảy ra).


3. Inference: Test với Inference có sẵn https://github.com/Syun1208/ocr-tma/tree/main/inference/en_ppstructure_mobile_v2.0_SLANet_infer

(pretrained_model gồm có Infernce và ... xem nhiều model đã train tại đây 
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/doc/doc_en/algorithm_overview_en.md#12   )

Test model đã train cũng ko vượt qua được PaddleOCR, Inference của những ông đã tham gia cuộc thi thì họ ko public. => Tự lấy data về train thui. 


- Anh Tú hd: 
    - Tạo 1 thư mục Paddle_OCR, git clone https://github.com/PaddlePaddle/PaddleOCR về
    - Tạo 1 file paddleocr_main.py
    - run cmd trên thư mục Padd_OCR: `python paddleocr_main.py --source <img_path>`

- I convert .py to .ipynb: I remove the argument parsing part and simply define the source directly within the code. 

```python

# anh Tú, paddleocr_main.py
'''
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image
import sys
import argparse
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter lang as ch, en, french, german, korean, japan
# to switch the language model in order.

def parse_arguments() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description = 'PaddleOCR')
    
    parser.add_argument(
        "--source", 
        default="0", 
        type=str
    )
    args = parser.parse_args()
    return args

def main():
    args = parse_arguments()
    ocr = PaddleOCR(use_angle_cls=True, lang='vi') # need to run only once to download and load model into memory
    img_path = args.source
    results = ocr.ocr(img_path, cls=True)
    text_recog = []
    confidence = 0
    count = 0
    for result in results:
        # print(result)
        text_recog.append(result[1][0])
        confidence += result[1][1]
        count+=1

    text = ' '.join(text_recog)
    confidence = confidence /count

    print(text, round(confidence,2))

if __name__ == '__main__':
    main()
    sys.exit(0)

# cách chạy là : python main.py --source <pathtoimage>
# pathtoimage la cai duong dan vao cai file hinh text
# python main.py --source cropped_image_0.9489.jpg
'''

from paddleocr import PaddleOCR
import argparse

def parse_arguments():
    parser = argparse.ArgumentParser(description='PaddleOCR')
    parser.add_argument("--source", default="0", type=str)
    args = parser.parse_args()
    return args

def main():
    args = parse_arguments()
    ocr = PaddleOCR(use_angle_cls=True, lang='vi')  # need to run only once to download and load model into memory
    img_path = args.source
    results = ocr.ocr(img_path, cls=True)

    text_recog = []
    total_confidence = 0
    count = 0

    for line in results:
        for result in line:
            # result is a tuple (box, (text, confidence))
            text = result[1][0]
            confidence = result[1][1]
            text_recog.append(text)
            total_confidence += confidence  # Accumulate the confidences
            count += 1

    text = ' '.join(text_recog)
    average_confidence = total_confidence / count if count else 0  # Calculate the average

    print(text, round(average_confidence, 2))

if __name__ == '__main__':
    main()
# python newmain_aTu.py --source cropped_image_0.9489.jpg

```

- Tạo 1 folder, clone PaddleOCR về. 
- Trong folder đó tạo 1 file main.py và run trong terminal của folder đã tạo lệnh 
python main.py --source <pathtoimage>

python main.py --source ocr\cropped_image_0.9489.jpg
(nó gắn link ảnh vào source)

- Output thu được: 
D:\OneDrive - codemely\Proj1_NhậnDien\code\code\roadapp\roadapp\ocr>python newmain.py --source cropped_image_0.9489.jpg
[2023/11/08 14:59:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Administrator/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Administrator/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='C:\\Users\\Administrator\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\paddleocr\\ppocr\\utils\\dict\\latin_dict.txt', use_space_char=True, vis_font_path='./doc/fonts/simfang.ttf', drop_score=0.5, e2e_algorithm='PGNet', e2e_model_dir=None, e2e_limit_side_len=768, e2e_limit_type='max', e2e_pgnet_score_thresh=0.5, e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_pgnet_valid_set='totaltext', e2e_pgnet_mode='fast', use_angle_cls=True, cls_model_dir='C:\\Users\\Administrator/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_image_shape='3, 48, 192', label_list=['0', '180'], cls_batch_num=6, cls_thresh=0.9, enable_mkldnn=False, cpu_threads=10, use_pdserving=False, warmup=False, sr_model_dir=None, sr_image_shape='3, 32, 128', sr_batch_num=1, draw_img_save_dir='./inference_results', save_crop_res=False, crop_res_save_dir='./output', use_mp=False, total_process_num=1, process_id=0, benchmark=False, save_log_path='./log_output/', show_log=True, use_onnx=False, output='./output', table_max_len=488, table_algorithm='TableAttn', table_model_dir=None, merge_no_span_structure=True, table_char_dict_path=None, layout_model_dir=None, layout_dict_path=None, layout_score_threshold=0.5, layout_nms_threshold=0.5, kie_algorithm='LayoutXLM', ser_model_dir=None, re_model_dir=None, use_visual_backbone=True, ser_dict_path='../train_data/XFUND/class_list_xfun.txt', ocr_order_method=None, mode='structure', image_orientation=False, layout=True, table=True, ocr=True, recovery=False, use_pdf2docx_api=False, invert=False, binarize=False, alphacolor=(255, 255, 255), lang='vi', det=True, rec=True, type='ocr', ocr_version='PP-OCRv4', structure_version='PP-StructureV2')
[2023/11/08 14:59:10] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.238084077835083
[2023/11/08 14:59:10] ppocr DEBUG: cls num  : 4, elapsed : 0.2565574645996094
[2023/11/08 14:59:11] ppocr DEBUG: rec_res num  : 4, elapsed : 1.2052311897277832
DUONG NGO O MINH DUONG 0.96



# ***1. Run nhanh PaddleOCR***

## Phân tích Code 

In [12]:
# !pip install paddleocr

# Import Lib
from paddleocr import PaddleOCR

# Tạo đối tượng ocr_model 
ocr_model = PaddleOCR(use_angle_cls=True, lang='en') # use_angle phân loại góc, Sử dụng 'en' cho tiếng Anh, 'ch' cho tiếng Trung, v.v.

# img_path
img_path = r'cropped_image_0.9489.jpg'      # cropped_img cùng cấp bậc với file .ipynb hiện tại, còn RESEARCHMODEL chứa 3.Paddleocr chứa .ipynb và ảnh crop thì dùng Relative ko được

# Thực hiện ocr_model.ocr() method trên hình ảnh, thu được result gồm cả 
results_list = ocr_model.ocr(img_path)
print(results_list)

# text_recog_list = []
# for line in results_list:
#     print('-each line', line)
#     for result in line:
#         print('--each result', result)
#         print('---result[1]', result[1])
#         text = result[1][0]
#         print('----each text', text)
#         text_recog_list.append(text)
# print('Text recognition is: ',' '.join(text_recog_list))

text_recog = ' '.join([result[1][0] for line in results_list for result in line])
print(text_recog)


[2023/11/22 02:38:09] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Administrator/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Administrator/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num

FileNotFoundError: [Errno 2] No such file or directory: 'cropped_image_0.9489.jpg'

## Đóng gói def, 1 click Run nhanh 1 ***IMAGE*** = Paddle OCR cơ bản

In [ ]:
# # Đóng gói def  bản gốc
# def ocr_from_image(img_path):
#     ocr = PaddleOCR(use_angle_cls=True, lang='vi')  # Initialize the PaddleOCR. This may download models if not already present.
#     results = ocr.ocr(img_path, cls=True)

#     text_recog = []
#     total_confidence = 0
#     count = 0

#     # Process results
#     for line in results:
#         for result in line:
#             text = result[1][0]
#             confidence = result[1][1]
#             text_recog.append(text)
#             total_confidence += confidence
#             count += 1

#     # Join recognized text and calculate average confidence
#     text = ' '.join(text_recog)
#     average_confidence = total_confidence / count if count else 0

#     return text, round(average_confidence, 2)

# Đóng gói def bản OPTIMIZE
def ocr_from_image(img_path):
    results = ocr_model.ocr(img_path, cls=True)
    extracted_texts = ' '.join([res[1][0] for line in results for res in line])
    average_confidence = sum(result[1][1] for line in results for result in line) / len(results)

    return extracted_texts, round(average_confidence, 2)

# Import Lib
from paddleocr import PaddleOCR

# Tạo đối tượng ocr_model 
# ocr_model = PaddleOCR(use_angle_cls=True, lang='en') # Sử dụng 'en' cho tiếng Anh, 'ch' cho tiếng Trung, v.v.
ocr = PaddleOCR(use_angle_cls=True, lang='vi') # Sử dụng 'en' cho tiếng Anh, 'ch' cho tiếng Trung, v.v.
img_path = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\sign_test_imgs\s1.jpg'

print(ocr_from_image(img_path))



## Đóng gói def, 1 click Run nhanh 1 ***FOLDER*** = Paddle OCR cơ bản

In [ ]:
# Thao tác với file
import os
import csv
from paddleocr import PaddleOCR


# Hàm tối ưu hóa OCR
def ocr_from_image(img_path):
    results = ocr_engine.ocr(img_path, cls=True)
    if results and results != [None]:    # Bổ sung trường hợp results là [None]   (type: list)
        extracted_texts = ' '.join([res[1][0] for line in results for res in line])
        return extracted_texts
    else: 
        return "Không thể nhận dạng"

def ocr_from_folderdata(ocr_engine, img_folder, output_csv):
    # Mở file CSV để ghi
    with open(output_csv, mode='a', newline='', encoding='utf-8') as file:   # add để ghi thêm, w viết lại tử đầu
        writer = csv.writer(file)
        writer.writerow(['Image Name', 'OCR Result'])     # Ghi hàng tiêu đề

        # Duyệt qua tất cả các file trong thư mục
        # for image_name in os.listdir(img_folder):
        #     if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        for image_name in [f for f in os.listdir(img_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]:
            
            img_path = os.path.join(img_folder, image_name)  # Tạo Đường dẫn đầy đủ đến hình ảnh, nhớ img_folder

            text_result = ocr_from_image(img_path, ocr)
            print(text_result)
            writer.writerow([image_name, text_result])
            print(f"Đã xử lý: {image_name}")

    print(f"All images processed. Results saved to {output_csv}")



In [ ]:
# Cấu hình PaddleOCR
ocr_model = PaddleOCR(use_angle_cls=True, lang='vi')

# Input folder và output .csv file
img_folder = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\crop_sign_test_imgs\\'
output_csv = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mecode\ResearchModel\DATA_TestResearch\output_test.csv'

ocr_from_folderdata(ocr_model, img_folder, output_csv)

### Thử 1 số model khác của PaddleOCR*** 
(with đổi pretrain models và font latin, kết quả thu được vẫn ko thay đổi gì)
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/doc/doc_en/algorithm_overview_en.md#12

- **Trained Model**: Sử dụng cho mục đích nghiên cứu, phát triển thêm, hoặc tinh chỉnh mô hình trên tập dữ liệu mới.
- **Inference Model**: Sử dụng cho việc triển khai thực tế, như trong các ứng dụng di động, web, hoặc các hệ thống tự động hóa khác.

`PaddleOCR` is a powerful optical character recognition (OCR) tool that can be used to extract text from images. The `PaddleOCR` function comes with various parameters that allow you to customize its behavior to fit different use cases. Some of the other parameters you can use with the `PaddleOCR` function:

1. **`use_gpu`**: 
2. **`use_angle_cls`**: enables the detection of text orientation, which can be useful for images with text in different orientations.
3. **`lang`**:  For example, `'vi'` is for Vietnamese. PaddleOCR supports multiple languages,
4. **`det_model_dir`**, **`rec_model_dir`**, **`cls_model_dir`**: These parameters allow you to specify custom paths for the detection model, recognition model, and classification model respectively. It's useful if you have trained your own models or want to use specific versions provided by PaddleOCR.
5. **`det_db_thresh`**, **`det_db_box_thresh`**, **`det_db_unclip_ratio`**, **`use_dilation`**
9. **`rec_char_type`**: Defines the type of characters expected in the recognition process, like `'ch'` for Chinese, `'en'` for English, etc.
10. **`rec_char_dict_path`**: Custom path for the dictionary file used in the recognition process.
11. **`max_text_length`**: Maximum length of the text to be recognized. This is useful for limiting the recognition process to texts of certain lengths.
12. **`rec_batch_num`**: Batch size for recognition. Adjusting this can impact the speed and memory usage during the OCR process.
13. **`rec_image_shape`**: Sets the shape of the input image for the recognition model. Changing this can impact the recognition accuracy for different sizes of texts.


In [ ]:
# Thao tác với file
import os
import csv
from paddleocr import PaddleOCR



# Hàm tối ưu hóa OCR
def ocr_from_image(img_path, ocr_engine):
    results = ocr_engine.ocr(img_path, cls=True)
    if results and results != [None]:    # Bổ sung trường hợp results là [None]   (type: list)
        extracted_texts = ' '.join([res[1][0] for line in results for res in line])
        return extracted_texts
    else: 
        return "Không thể nhận dạng"


def ocr_from_image(img_folder, output_csv):
    # Mở file CSV để ghi
    with open(output_csv, mode='a', newline='', encoding='utf-8') as file:   # add để ghi thêm, w viết lại tử đầu
        writer = csv.writer(file)
        writer.writerow(['Image Name', 'OCR Result'])     # Ghi hàng tiêu đề

        # Duyệt qua tất cả các file trong thư mục
        # for image_name in os.listdir(img_folder):
        #     if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        for image_name in [f for f in os.listdir(img_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]:
            
            img_path = os.path.join(img_folder, image_name)  # Tạo Đường dẫn đầy đủ đến hình ảnh

            text_result = ocr_from_image(img_path, ocr)
            print(text_result)
            writer.writerow([image_name, text_result])
            print(f"Đã xử lý: {image_name}")

    print(f"All images processed. Results saved to {output_csv}")



# Cấu hình PaddleOCR
# ocr = PaddleOCR(use_angle_cls=True, lang='vi')
# From: https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/doc/doc_en/algorithm_overview_en.md#12
ocr = PaddleOCR(use_angle_cls=True, lang='vi',
                det_model_dir=r'3.Paddleocr\trained_model\det_r50_vd_sast_icdar15_v2.0_train', # dêtction - Backbone ResNet50_vd - Model SAST on  ICDAR2015 dataset
                rec_model_dir=r'3.Paddleocr\trained_model\rec_r45_abinet_train'    # recognition - Backbone ResNet45 - Model Abinet
                ,rec_char_dict_path= r'D:\Hoc IT\Doan Ngoc Cuong\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mycode\ResearchModel\4.Paddleocr_PaddlePaddle\4.4.PaddleOCR_DNC\ppocr\utils\dict\latin_dict.txt'
                # ,rec_char_type='latin'   
                # đặt trỏ vào từ điển latin còn được, chứ trỏ vào từ điển tiếng việt output siêu tệ vì chưa được train
                ) 

# Input folder và output .csv file
img_folder = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mycode\ResearchModel\DATA_TestResearch\crop_sign_test_imgs'
output_csv = r'D:\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mycode\ResearchModel\DATA_TestResearch\TEST_paddle_results.csv'
ocr_from_image(ocr, img_folder, output_csv)

- Thử với ký tự Latin: https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/doc/doc_en/models_list_en.md#23-multilingual-recognition-modelupdating
```ocr = PaddleOCR(use_angle_cls=True, lang='vi',
                det_model_dir=r'3.Paddleocr\trained_model\det_r50_vd_sast_icdar15_v2.0_train', # dêtction - Backbone ResNet50_vd - Model SAST on  ICDAR2015 dataset
                rec_model_dir=r'3.Paddleocr\trained_model\rec_r45_abinet_train')    # recognition - Backbone ResNet45 - Model Abinet
```
=> All the same output lúc chưa chỉ định cụ thể model


## 1 Số Bug và Kết luận: 

1. When pip install paddle, bug: 

```
ModuleNotFoundError: No module named 'common'
      [end of output]

process, and is likely not a problem with pip.
error: subprocess-exited-with-error     

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

```
=> yêu cầu cần cài PyMuPDF


- Lỗi gặp phải khi cài PyMuPDF

```ERROR: Failed building wheel for PyMuPDF
Failed to build PyMuPDF
ERROR: Could not build wheels for PyMuPDF, which is required to install pyproject.toml-based projects
```
Fix:
- Link fix: https://stackoverflow.com/questions/76379293/how-can-i-fix-the-error-in-pymupdf-when-installing-paddleocr-with-pip?fbclid=IwAR2LFKySkD65hEZmYHqLRtw49cS06tKH92H7CA3bWAORlqjL3WMrNPzx_0E
(down drive sau: https://drive.google.com/drive/folders/1PESjDkovpvnrWFTKji4-qgT3rcVz-o-F - ko hiểu sao link lúc down được lúc ko)

- pip install path to the whl file (`Relative Path`)

pip install "D:\Hoc IT\Doan Ngoc Cuong\OneDrive - codemely\Proj1_NhanDienBienBaoTenDuong_CODEMELY\mycode\ResearchModel\3.Paddleocr\PyMuPDF-1.20.2-cp311-cp311-win_amd64.whl"

2. When pip install paddleocr: 

```
Exception: Building py-lmdb from source on Windows requires the "patch-ng" python module.
      [end of output]

  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error     

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
```


https://github.com/PaddlePaddle/PaddleOCR
https://github.com/Mushroomcat9998/PaddleOCR/tree/main 
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.6/doc/doc_en/quickstart_en.md#1-installation

- Thực tế test cho thấy, các biển tiếng anh PaddleOCR tốt hơn OCR, easyocr. 
- Cơ mà sao set lang = 'vi' rồi mà PaddleOCR vẫn ko nhận dạng được chữ tiếng việt